In [1]:
import os

import numpy as np  
import pandas as pd  
import scipy.optimize as opt
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torch.autograd import Variable

## nn.Module is base class
## LogisticRegressionModel is actually same as LinearRegressionModel. Cost function makes all the difference
## Naming is just to show what we are doing
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size,output_size):
        ##inherit everything from nn.module
        super(LogisticRegressionModel,self).__init__()
        self.linear=nn.Linear(input_size,output_size)
        
    def forward(self, x):
        out=self.linear(x)
        return out

In [4]:
df = pd.read_csv('Data/data.csv')
df.iloc[:, -1] = LabelEncoder().fit_transform(df.iloc[:, -1])
df = df.astype(np.uint8)
df.head()

,pixel_0000,pixel_0001,pixel_0002,pixel_0003,pixel_0004,pixel_0005,pixel_0006,pixel_0007,pixel_0008,pixel_0009,...,pixel_1015,pixel_1016,pixel_1017,pixel_1018,pixel_1019,pixel_1020,pixel_1021,pixel_1022,pixel_1023,character
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
numberOfClasses = 46
img_width = 32
img_height = 32
pixel_lenght=img_width*img_height
img_depth = 1

## We have 4 feature
input_size=pixel_lenght
output_size=numberOfClasses
#Instantiate model
model= LogisticRegressionModel(input_size,output_size)

#Instantiate Loss Class
##criterion=nn.MSELoss() this is for linear regression
#This is for logistic regression
criterion=nn.CrossEntropyLoss()

# Instantiate Optimizer Class
learning_rate=0.005
momentum=0.95
# Optimizer supports momentum.
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate,momentum=momentum)

In [75]:
#splitting data for training and testing purposes
from sklearn.cross_validation import train_test_split

x=df.iloc[:, :-1].as_matrix()
y=df.iloc[:, -1].as_matrix()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

x_train=x_train.astype(np.float32)
x_test=x_test.astype(np.float32)
y_train=y_train.astype(np.int64)
y_test=y_test.astype(np.int64)


labels = Variable(torch.from_numpy(y_train))

In [78]:
# Lets train our model
epochs=50

for epoch in range(epochs):
    epoch +=1
    #Convert numpy array to torch Variable
    inputs = Variable(torch.from_numpy(x_train))
    labels = Variable(torch.from_numpy(y_train))
    
    #Clear gradients w.r.t. parameters
    optimizer.zero_grad()
    
    #Forward to get output
    outputs=model(inputs)
    
    #Calculate Loss
    loss=criterion(outputs,labels)
    
    #Getting gradients w.r.t parameters
    loss.backward()
    
    #Updating parameters
    optimizer.step()
    
    if(epoch%10==0):
        print("epoch {}, loss {}".format(epoch,loss.data[0]))

epoch 10, loss 236.073120117
epoch 20, loss 179.401809692
epoch 30, loss 164.358703613
epoch 40, loss 133.035400391
epoch 50, loss 127.367156982


In [79]:
predicted = model(Variable(torch.from_numpy(x_test))).data.numpy()
#multi-class regression: for a given row column with largest value's index represents predicted character 
#due to lack of time I didnt make the conversion however results look true

maxIndexes=predicted.argmax(1)



(y_test != maxIndexes).sum()/float(y_test.size)

0.30123188405797102

In [50]:
print(maxIndexes.len);
print(y_test.len);

AttributeError: 'numpy.ndarray' object has no attribute 'len'